# Installation librairies

# Import librairies

In [ ]:
import pandas as pd
import os

# DATA IMPORT:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# chemins
path = '/content/drive/MyDrive/ICU/'
path_down = '/content/drive/MyDrive/ICU/Download_data/'
path_analyse = '/content/drive/MyDrive/ICU/Analyse/'
path_carto = '/content/drive/MyDrive/ICU/Carto/'

# Import Dataset

In [ ]:
# import Stations data csv
df = pd.read_csv(path_analyse+'Stations_All.csv',sep=';', low_memory=False)

In [ ]:
display(df.info())
display(df.head(2))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4900627 entries, 0 to 4900626
Data columns (total 14 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   station_name                                object 
 1   heure_utc                                   object 
 2   id                                          int64  
 3   type_de_station                             object 
 4   temperature                                 float64
 5   humidite                                    float64
 6   pression                                    float64
 7   pluie                                       float64
 8   pluie_intensite_max                         float64
 9   direction_du_vecteur_vent_moyen             float64
 10  force_moyenne_du_vecteur_vent               float64
 11  direction_du_vecteur_de_vent_max            float64
 12  direction_du_vecteur_de_rafale_de_vent_max  float64
 13  force_rafale_max           

None

,station_name,heure_utc,id,type_de_station,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max
0,00-station-meteo-toulouse-valade,2020-06-08 13:15:00+00:00,0,ISS,16.1,74.0,99500.0,0.0,0.0,0.0,0.0,6.0,135.0,3.0
1,00-station-meteo-toulouse-valade,2020-06-08 13:30:00+00:00,0,ISS,17.0,71.0,99500.0,0.0,0.0,58.0,2.0,3.0,67.5,8.0


# Transformation & select 2023 data

In [ ]:
# transforme la colonne 'date' au format datetime
df['heure_utc'] = pd.to_datetime(df['heure_utc'], utc=True)

# format la colonne 'id' en string
df['id'] = df['id'].astype(str)

# Filtrer les lignes où l'année est = à 2023
df_2023 = df[df['heure_utc'].dt.year == 2023].sort_values(by=['id','heure_utc'])

# Filtrer les valeurs de températures aberrante

df_2023 = df_2023[df_2023['temperature']> -20]

df_2023.info()

<class 'pandas.core.frame.DataFrame'>
Index: 934919 entries, 95180 to 902386
Data columns (total 14 columns):
 #   Column                                      Non-Null Count   Dtype              
---  ------                                      --------------   -----              
 0   station_name                                934919 non-null  object             
 1   heure_utc                                   934919 non-null  datetime64[ns, UTC]
 2   id                                          934919 non-null  object             
 3   type_de_station                             916871 non-null  object             
 4   temperature                                 934919 non-null  float64            
 5   humidite                                    934919 non-null  float64            
 6   pression                                    934919 non-null  float64            
 7   pluie                                       934919 non-null  float64            
 8   pluie_intensite_max      

# Resample sur un intervalle de 3 heures

In [ ]:
# Fixer 'heure_utc' comme index
df_2023.set_index('heure_utc', inplace=True)

# Séparer les colonnes numériques et non numériques (sans 'id' pour les non-numériques)
cols_num = df_2023.select_dtypes(include=['number']).columns
cols_non_num = df_2023.select_dtypes(exclude=['number']).columns.difference(['id'])

# Resample uniquement les colonnes numériques sur un intervalle de 3 heures
df_numeric_resampled = df_2023[cols_num].groupby(df_2023['id']).resample('3h').mean(numeric_only=True)

# Récupérer les colonnes non numériques (comme id) sans affecter le resampling
df_non_numeric = df_2023[cols_non_num].groupby(df_2023['id']).resample('3h').first()

# Fusionner les résultats numériques et non numériques
df_2023_3h = pd.concat([df_numeric_resampled, df_non_numeric], axis=1).reset_index()

In [ ]:
df_2023_3h.head()

,id,heure_utc,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max,station_name,type_de_station
0,1,2023-01-01 00:00:00+00:00,13.483333,80.750000,100200.000000,0.0,0.0,41.000000,13.250000,2.000000,45.000000,25.333333,01-station-meteo-toulouse-meteopole,ISS
1,1,2023-01-01 03:00:00+00:00,13.381818,82.545455,100136.363636,0.0,0.0,44.000000,11.818182,2.090909,47.045455,24.909091,01-station-meteo-toulouse-meteopole,ISS
2,1,2023-01-01 06:00:00+00:00,13.600000,81.000000,100100.000000,0.0,0.0,39.000000,11.500000,1.000000,22.500000,29.000000,01-station-meteo-toulouse-meteopole,ISS
3,1,2023-01-01 09:00:00+00:00,15.183333,78.250000,100075.000000,0.0,0.0,37.333333,19.333333,1.416667,31.875000,32.583333,01-station-meteo-toulouse-meteopole,ISS
4,1,2023-01-01 12:00:00+00:00,15.058333,78.333333,99916.666667,0.0,0.0,43.666667,19.000000,2.000000,45.000000,32.416667,01-station-meteo-toulouse-meteopole,ISS


In [ ]:
# Groupby sur 'id' puis compter les occurrences des valeurs de 'temperature'
temperature_counts = df_2023_3h.groupby('id')['temperature'].count().sort_values(ascending=False)

#seuil 2628
seuil = 2569

# Filtre les stations ayant de résultats (8 mesures par jour sur 365 jours x 0.88 = 2570)
stations_to_keep = temperature_counts[temperature_counts >= seuil ].index

df_stations_to_keep = df_2023_3h[df_2023_3h['id'].isin(stations_to_keep)]

In [ ]:
display(temperature_counts)

,temperature
id,
51,2916
1,2915
2,2915
11,2911
4,2909
24,2904
13,2900
17,2898
28,2826


In [ ]:
#list des id de stations + count
df_stations_to_keep['id'].unique(), df_stations_to_keep['id'].nunique()

(array(['1', '11', '13', '14', '17', '19', '2', '22', '24', '26', '27',
        '28', '31', '4', '51', '62', '63', '64'], dtype=object),
 18)

# Calcul du delta de T° avec la station ref

In [ ]:
# 1. Extraire les lignes de la station référence id = 31
df_ref = df_stations_to_keep[df_stations_to_keep['id'] == '31']

# 2. Fusionner les lignes de référence avec le reste sur la base de la colonne 'timestamp'
df_merged = df_stations_to_keep.merge(df_ref[['heure_utc', 'temperature']], on='heure_utc', suffixes=('', '_ref'))

# 3. Calculer le delta de température
df_merged['delta_temperature'] = df_merged['temperature'] - df_merged['temperature_ref']

# 4. Filtrer pour ne conserver que les lignes où id != 1 (puisque id = 1 serait delta = 0)
df_delta = df_merged[df_merged['id'] != 1]

# Afficher le résultat
display(df_delta[['id', 'heure_utc', 'temperature', 'temperature_ref', 'delta_temperature']].head(5))

,id,heure_utc,temperature,temperature_ref,delta_temperature
0,1,2023-01-01 00:00:00+00:00,13.483333,12.141667,1.341667
1,11,2023-01-01 00:00:00+00:00,12.450000,12.141667,0.308333
2,13,2023-01-01 00:00:00+00:00,12.458333,12.141667,0.316667
3,19,2023-01-01 00:00:00+00:00,11.966667,12.141667,-0.175000
4,2,2023-01-01 00:00:00+00:00,12.591667,12.141667,0.450000


In [ ]:
# Compter le nombre de fois où delta_temperature est supérieur à 4 pour chaque id
delta_count = df_delta[df_delta['delta_temperature'] >= 4].groupby('id').size().reset_index(name='compteur')
#display(delta_count)

# Fusionner ce comptage avec le DataFrame d'origine
df_with_compteur = df_delta.merge(delta_count, on='id', how='left')

# Compter le nombre total d'occurrences pour chaque id dans le DataFrame d'origine
total_count = df_delta[~df_delta['temperature'].isnull()].groupby('id').size().reset_index(name='total')

# Remplir les NaN dans la colonne 'compteur' avec 0 pour les ids qui n'ont aucun delta supérieur à 4
df_with_compteur['compteur'] = df_with_compteur['compteur'].fillna(0).astype(int)

# Fusionner ce comptage avec le DataFrame d'origine
df_with_compteur_total = df_with_compteur.merge(total_count, on='id', how='left')

#Ratio
df_with_compteur_total['Ratio'] = df_with_compteur_total['compteur']/df_with_compteur_total['total']

# Afficher le DataFrame mis à jour
display(df_with_compteur_total.head(2))

,id,heure_utc,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max,station_name,type_de_station,temperature_ref,delta_temperature,compteur,total,Ratio
0,1,2023-01-01 00:00:00+00:00,13.483333,80.75,100200.0,0.0,0.0,41.0,13.25,2.0,45.0,25.333333,01-station-meteo-toulouse-meteopole,ISS,12.141667,1.341667,657,2915,0.225386
1,11,2023-01-01 00:00:00+00:00,12.450000,91.60,100400.0,0.0,0.0,0.0,1.90,4.4,99.0,13.700000,11-station-meteo-toulouse-soupetard,ISS,12.141667,0.308333,39,2911,0.013397


In [ ]:
# Sélectionner uniquement les colonnes numériques pour calculer la moyenne
Top_3 = df_with_compteur_total.groupby('id').mean(numeric_only=True).sort_values('Ratio', ascending=False).head(4)
Less_3 = df_with_compteur_total.groupby('id').mean(numeric_only=True).sort_values('Ratio', ascending=False).tail(4)

# Afficher les résultats
display(Top_3)
display(Less_3)

,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max,temperature_ref,delta_temperature,compteur,total,Ratio
id,,,,,,,,,,,,,,,
1,15.989090,67.764575,99436.668863,0.009690,0.005513,114.748844,4.708773,5.183682,116.632845,10.328753,14.837124,1.501774,657.0,2915.0,0.225386
28,16.015001,72.413514,99174.441233,0.000000,0.007044,41.652309,1.732589,5.341875,120.192198,7.384526,14.837124,1.617669,310.0,2826.0,0.109696
27,15.464491,50.287860,99911.146485,0.001601,0.007159,12.582500,1.720719,4.849287,109.108947,6.056168,14.837124,1.643704,272.0,2726.0,0.099780
26,15.393273,75.312857,100123.413289,0.014333,0.007834,138.298163,2.960641,6.238703,140.370815,10.862691,14.837124,1.147570,186.0,2600.0,0.071538


,temperature,humidite,pression,pluie,pluie_intensite_max,direction_du_vecteur_vent_moyen,force_moyenne_du_vecteur_vent,direction_du_vecteur_de_vent_max,direction_du_vecteur_de_rafale_de_vent_max,force_rafale_max,temperature_ref,delta_temperature,compteur,total,Ratio
id,,,,,,,,,,,,,,,
4,15.071461,77.400243,99903.392141,0.047625,0.011048,70.257829,1.656507,4.704480,105.850793,6.125844,14.837124,0.699361,39.0,2909.0,0.013407
11,15.266508,75.987208,99895.171823,0.000000,0.007866,4.246483,0.460448,4.217472,94.893126,5.397197,14.837124,0.960492,39.0,2911.0,0.013397
62,15.376253,59.281700,100272.082519,0.013317,0.006902,161.696205,3.900841,7.355402,165.496554,11.470405,14.837124,0.640783,9.0,2622.0,0.003432
31,14.837124,63.176508,99954.080137,0.012640,0.006378,119.696615,3.972199,5.673027,127.643112,10.929285,14.837124,0.000000,0.0,2569.0,0.000000


In [ ]:
# Exporter dataset_ICU en un fichier CSV
output_path = os.path.join(path_analyse, 'dataset_ICU.csv')  # Spécifier le chemin de sauvegarde du fichier CSV
df_with_compteur_total.to_csv(output_path, sep=';', index=False)

print(f"DataFrame exporté en CSV à : {output_path}")

DataFrame exporté en CSV à : /content/drive/MyDrive/ICU/Analyse/dataset_ICU.csv
